# Advanced Probability Distributions Tutorial

This notebook teaches advanced probability concepts and distributions.

We will cover:
1. Exponential Distribution
2. Gamma Distribution
3. Beta Distribution
4. Central Limit Theorem
5. Maximum Likelihood Estimation

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import gamma as gamma_function
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Exponential Distribution

### What is the Exponential Distribution?

The exponential distribution models the time between independent events.

**Key concept:** If events occur at a constant average rate, the waiting time until the next event follows an exponential distribution.

**Examples:**
- Time between customer arrivals
- Time until next phone call
- Time between machine failures
- Time between goals in a game

### Rate Parameter λ (Lambda)

The exponential distribution has one parameter:
- **λ (lambda)** = rate of events per unit time
- Higher λ means events happen more frequently
- Higher λ means shorter average waiting time

**Relationship:**
- If events occur at rate λ per hour
- Then average time between events = 1/λ hours

### PDF and CDF

**Probability Density Function (PDF):**

$$f(x) = \lambda e^{-\lambda x}$$

For x ≥ 0

**Cumulative Distribution Function (CDF):**

$$F(x) = 1 - e^{-\lambda x}$$

This gives P(X ≤ x), the probability of waiting time x or less.

**Mean:** $\mu = \frac{1}{\lambda}$

**Variance:** $\sigma^2 = \frac{1}{\lambda^2}$

### Example: Customer Arrivals

**Problem:** Customers arrive at a rate of 3 per hour (λ = 3). What is the probability the next customer arrives within 20 minutes (1/3 hour)?

**Given:**
- λ = 3 customers per hour
- We want P(X ≤ 1/3)

In [ ]:
# Example: Customer arrivals
print("Example: Customer Arrivals")
print("=" * 40)

lambda_rate = 3  # Arrivals per hour
time = 1/3       # 20 minutes = 1/3 hour

# Calculate probability using CDF
prob = stats.expon.cdf(time, scale=1/lambda_rate)

print(f"Rate: λ = {lambda_rate} customers/hour")
print(f"Time: {time:.3f} hours (20 minutes)")
print(f"\nP(X ≤ {time:.3f}) = {prob:.4f}")
print(f"P(X ≤ {time:.3f}) = {prob:.2%}")

# Calculate mean and variance
mean = 1 / lambda_rate
variance = 1 / (lambda_rate ** 2)
std = np.sqrt(variance)

print(f"\nMean waiting time: {mean:.3f} hours ({mean*60:.1f} minutes)")
print(f"Standard deviation: {std:.3f} hours ({std*60:.1f} minutes)")

# Probability of waiting more than 30 minutes
prob_gt_30 = 1 - stats.expon.cdf(0.5, scale=1/lambda_rate)
print(f"\nP(X > 0.5 hours) = {prob_gt_30:.4f}")

In [ ]:
# Plot PDF for different λ values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.linspace(0, 5, 1000)
lambda_values = [0.5, 1, 2, 3]

# Plot PDFs
for lam in lambda_values:
    pdf = stats.expon.pdf(x, scale=1/lam)
    ax1.plot(x, pdf, linewidth=2, label=f'λ={lam}')

ax1.set_xlabel('Time (x)')
ax1.set_ylabel('Probability Density')
ax1.set_title('Exponential PDF for Different λ')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot CDFs
for lam in lambda_values:
    cdf = stats.expon.cdf(x, scale=1/lam)
    ax2.plot(x, cdf, linewidth=2, label=f'λ={lam}')

ax2.set_xlabel('Time (x)')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Exponential CDF for Different λ')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Higher λ means:")
print("  - Steeper PDF (events happen faster)")
print("  - CDF rises faster (shorter waiting times)")

In [ ]:
# Simulation: Exponential waiting times
print("Simulation: Exponential Waiting Times")
print("=" * 40)

lambda_rate = 2  # Events per hour
n_events = 1000

# Simulate waiting times
waiting_times = np.random.exponential(1/lambda_rate, n_events)

print(f"Simulating {n_events} waiting times with λ = {lambda_rate}")
print(f"\nSimulated mean: {np.mean(waiting_times):.4f} hours")
print(f"Theoretical mean: {1/lambda_rate:.4f} hours")
print(f"\nSimulated std: {np.std(waiting_times, ddof=1):.4f} hours")
print(f"Theoretical std: {1/lambda_rate:.4f} hours")

# Plot histogram with theoretical PDF
fig, ax = plt.subplots(figsize=(10, 6))

# Histogram
ax.hist(waiting_times, bins=50, density=True, alpha=0.7, 
        color='skyblue', edgecolor='black', label='Simulated data')

# Theoretical PDF
x_range = np.linspace(0, np.max(waiting_times), 1000)
pdf_theoretical = stats.expon.pdf(x_range, scale=1/lambda_rate)
ax.plot(x_range, pdf_theoretical, 'r-', linewidth=2, 
        label='Theoretical PDF')

# Add mean line
ax.axvline(1/lambda_rate, color='green', linestyle='--', linewidth=2, 
           label=f'Mean = {1/lambda_rate:.2f}')

ax.set_xlabel('Waiting Time (hours)')
ax.set_ylabel('Probability Density')
ax.set_title(f'Exponential Distribution: Simulated vs Theoretical (λ={lambda_rate})')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## 2. Gamma Distribution

### What is the Gamma Distribution?

The Gamma distribution models the sum of k independent exponential waiting times.

**Key concept:** If you wait for k events to occur, and each inter-arrival time is exponential with rate λ, then the total waiting time follows a Gamma distribution.

**Examples:**
- Time until the 5th customer arrives
- Time until 3 goals are scored
- Sum of several independent exponential variables

### Parameters and PDF

The Gamma distribution has two parameters. There are two common notations:

**Shape-Rate notation:**
- k (or α) = shape parameter (number of events)
- λ (or β) = rate parameter

**Shape-Scale notation:**
- k (or α) = shape parameter
- θ = scale parameter = 1/λ

**Probability Density Function (shape-rate):**

$$f(x) = \frac{\lambda^k x^{k-1} e^{-\lambda x}}{\Gamma(k)}$$

For x > 0

Where Γ(k) is the gamma function.

**Mean:** $\mu = \frac{k}{\lambda} = k\theta$

**Variance:** $\sigma^2 = \frac{k}{\lambda^2} = k\theta^2$

### Example: Waiting for Multiple Events

**Problem:** Customers arrive at rate λ = 2 per hour. What is the expected time until 5 customers arrive?

**Given:**
- k = 5 events
- λ = 2 per hour

In [ ]:
# Example: Waiting for multiple events
print("Example: Waiting for 5 Customers")
print("=" * 40)

k = 5           # Shape: number of events
lambda_rate = 2 # Rate: events per hour
theta = 1 / lambda_rate  # Scale parameter

# Calculate mean and variance
mean = k / lambda_rate
variance = k / (lambda_rate ** 2)
std = np.sqrt(variance)

print(f"Shape: k = {k} customers")
print(f"Rate: λ = {lambda_rate} per hour")
print(f"Scale: θ = {theta} hours")
print(f"\nExpected waiting time: {mean} hours")
print(f"Standard deviation: {std:.3f} hours")

# Probability of waiting less than 3 hours
prob = stats.gamma.cdf(3, a=k, scale=theta)
print(f"\nP(X < 3 hours) = {prob:.4f}")
print(f"P(X < 3 hours) = {prob:.2%}")

# 90th percentile
time_90 = stats.gamma.ppf(0.9, a=k, scale=theta)
print(f"\n90th percentile: {time_90:.2f} hours")
print(f"(90% of the time, 5 customers arrive within {time_90:.2f} hours)")

In [ ]:
# Plot Gamma PDF for different parameter settings
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.linspace(0, 20, 1000)

# Different shapes, same rate
theta = 1
for k in [1, 2, 5, 10]:
    pdf = stats.gamma.pdf(x, a=k, scale=theta)
    ax1.plot(x, pdf, linewidth=2, label=f'k={k}, θ={theta}')

ax1.set_xlabel('x')
ax1.set_ylabel('Probability Density')
ax1.set_title('Gamma Distribution: Different Shapes (k)')
ax1.legend()
ax1.grid(alpha=0.3)
ax1.set_xlim(0, 20)

# Same shape, different scales
k = 3
for theta in [0.5, 1, 2, 3]:
    pdf = stats.gamma.pdf(x, a=k, scale=theta)
    ax2.plot(x, pdf, linewidth=2, label=f'k={k}, θ={theta}')

ax2.set_xlabel('x')
ax2.set_ylabel('Probability Density')
ax2.set_title('Gamma Distribution: Different Scales (θ)')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.set_xlim(0, 20)

plt.tight_layout()
plt.show()

In [ ]:
# Simulation: Gamma as sum of exponentials
print("Simulation: Gamma as Sum of Exponentials")
print("=" * 40)

k = 4           # Wait for 4 events
lambda_rate = 2 # Rate per hour
n_simulations = 10000

# Method 1: Sum k exponential random variables
total_times_method1 = []
for _ in range(n_simulations):
    # Generate k exponential waiting times
    waiting_times = np.random.exponential(1/lambda_rate, k)
    # Sum them to get total time
    total_times_method1.append(np.sum(waiting_times))

total_times_method1 = np.array(total_times_method1)

# Method 2: Directly sample from Gamma
total_times_method2 = np.random.gamma(k, 1/lambda_rate, n_simulations)

print(f"Simulating waiting time for k={k} events, λ={lambda_rate}")
print(f"\nMethod 1 (sum of exponentials):")
print(f"  Mean: {np.mean(total_times_method1):.3f}")
print(f"  Std: {np.std(total_times_method1, ddof=1):.3f}")

print(f"\nMethod 2 (direct Gamma sampling):")
print(f"  Mean: {np.mean(total_times_method2):.3f}")
print(f"  Std: {np.std(total_times_method2, ddof=1):.3f}")

print(f"\nTheoretical values:")
print(f"  Mean: {k/lambda_rate:.3f}")
print(f"  Std: {np.sqrt(k)/lambda_rate:.3f}")

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(total_times_method1, bins=50, density=True, alpha=0.5, 
        color='blue', label='Sum of Exponentials')
ax.hist(total_times_method2, bins=50, density=True, alpha=0.5, 
        color='red', label='Direct Gamma')

# Theoretical PDF
x_range = np.linspace(0, 8, 1000)
pdf_theoretical = stats.gamma.pdf(x_range, a=k, scale=1/lambda_rate)
ax.plot(x_range, pdf_theoretical, 'k-', linewidth=2, 
        label='Theoretical Gamma PDF')

ax.set_xlabel('Total Waiting Time')
ax.set_ylabel('Probability Density')
ax.set_title(f'Gamma Distribution: Two Equivalent Methods (k={k}, λ={lambda_rate})')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Exercise: Basketball Possessions Model

In basketball, possessions occur roughly at a constant rate.

Let's model:
- Time between possessions (Exponential)
- Time until k possessions occur (Gamma)

In [ ]:
# Exercise: Basketball possessions
print("Exercise: Basketball Possessions Model")
print("=" * 40)

# Assume 100 possessions per 48-minute game
# Rate: 100/48 ≈ 2.08 possessions per minute
lambda_poss = 100 / 48

print(f"Rate: {lambda_poss:.2f} possessions per minute")
print(f"Mean time between possessions: {1/lambda_poss:.2f} minutes")

# Simulate a game (48 minutes)
np.random.seed(123)
game_duration = 48  # minutes
possession_times = []  # Times when possessions occur
current_time = 0

while current_time < game_duration:
    # Sample time until next possession (exponential)
    wait_time = np.random.exponential(1/lambda_poss)
    current_time += wait_time
    if current_time < game_duration:
        possession_times.append(current_time)

possession_times = np.array(possession_times)
n_possessions = len(possession_times)

print(f"\nSimulated {n_possessions} possessions in {game_duration} minutes")

# Calculate inter-possession times
inter_possession_times = np.diff([0] + list(possession_times))

print(f"\nInter-possession times:")
print(f"  Mean: {np.mean(inter_possession_times):.3f} minutes")
print(f"  Expected (Exponential): {1/lambda_poss:.3f} minutes")

# Model time until 10 possessions (Gamma)
k_poss = 10
time_to_k = possession_times[k_poss - 1]  # Time of 10th possession

print(f"\nTime until {k_poss} possessions: {time_to_k:.2f} minutes")
print(f"Expected (Gamma): {k_poss/lambda_poss:.2f} minutes")

# Fit Exponential and Gamma to the data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Exponential fit for inter-possession times
ax1.hist(inter_possession_times, bins=20, density=True, alpha=0.7, 
         color='skyblue', edgecolor='black', label='Simulated data')

x_exp = np.linspace(0, 3, 1000)
pdf_exp = stats.expon.pdf(x_exp, scale=1/lambda_poss)
ax1.plot(x_exp, pdf_exp, 'r-', linewidth=2, label='Exponential fit')

ax1.set_xlabel('Time Between Possessions (minutes)')
ax1.set_ylabel('Probability Density')
ax1.set_title('Exponential Model for Inter-Possession Times')
ax1.legend()
ax1.grid(alpha=0.3)

# Gamma model for time to k possessions
# Simulate many games and track time to 10th possession
time_to_10_samples = []
for _ in range(1000):
    times = np.cumsum(np.random.exponential(1/lambda_poss, 10))
    time_to_10_samples.append(times[9])  # 10th possession

ax2.hist(time_to_10_samples, bins=30, density=True, alpha=0.7, 
         color='lightgreen', edgecolor='black', label='Simulated data')

x_gamma = np.linspace(0, 15, 1000)
pdf_gamma = stats.gamma.pdf(x_gamma, a=10, scale=1/lambda_poss)
ax2.plot(x_gamma, pdf_gamma, 'r-', linewidth=2, label='Gamma fit')

ax2.set_xlabel('Time to 10 Possessions (minutes)')
ax2.set_ylabel('Probability Density')
ax2.set_title('Gamma Model for Time to k Possessions')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## 3. Beta Distribution

### What is the Beta Distribution?

The Beta distribution models probabilities restricted to the interval [0, 1].

**Key concept:** When you need a probability distribution for a probability (like success rate, shooting percentage, click-through rate), use the Beta distribution.

**Examples:**
- Shooting percentage in basketball
- Conversion rate in e-commerce
- Click-through rate in advertising
- Win probability

### Parameters and PDF

The Beta distribution has two parameters:
- **α (alpha)** = shape parameter (related to successes)
- **β (beta)** = shape parameter (related to failures)

**Probability Density Function:**

$$f(x) = \frac{x^{\alpha-1}(1-x)^{\beta-1}}{B(\alpha, \beta)}$$

For 0 ≤ x ≤ 1

Where B(α, β) is the beta function.

**Mean:** $\mu = \frac{\alpha}{\alpha + \beta}$

**Mode (when α, β > 1):** $\frac{\alpha - 1}{\alpha + \beta - 2}$

**Variance:** $\sigma^2 = \frac{\alpha\beta}{(\alpha + \beta)^2(\alpha + \beta + 1)}$

### Example 1: Symmetric Beta

**Problem:** A coin has unknown bias. We model the probability of heads with Beta(5, 5). What is the expected probability of heads?

**Given:**
- α = 5
- β = 5

In [ ]:
# Example 1: Symmetric Beta
print("Example 1: Symmetric Beta Distribution")
print("=" * 40)

alpha = 5
beta = 5

# Calculate mean
mean = alpha / (alpha + beta)

# Calculate mode
mode = (alpha - 1) / (alpha + beta - 2)

# Calculate variance
variance = (alpha * beta) / ((alpha + beta)**2 * (alpha + beta + 1))
std = np.sqrt(variance)

print(f"Parameters: α = {alpha}, β = {beta}")
print(f"\nMean (expected probability): {mean:.3f}")
print(f"Mode (most likely value): {mode:.3f}")
print(f"Standard deviation: {std:.3f}")

# Calculate probability that true probability > 0.6
prob = 1 - stats.beta.cdf(0.6, alpha, beta)
print(f"\nP(p > 0.6) = {prob:.4f}")

### Example 2: Skewed Beta

**Problem:** A shooter has made 15 shots and missed 5. Model their shooting percentage with Beta(16, 6). What is the 95% credible interval?

**Given:**
- α = 16 (successes + 1)
- β = 6 (failures + 1)

In [ ]:
# Example 2: Skewed Beta
print("Example 2: Skewed Beta Distribution")
print("=" * 40)

alpha = 16
beta = 6

# Calculate statistics
mean = alpha / (alpha + beta)
mode = (alpha - 1) / (alpha + beta - 2)

print(f"Parameters: α = {alpha}, β = {beta}")
print(f"\nMean shooting percentage: {mean:.3f} ({mean:.1%})")
print(f"Mode (most likely): {mode:.3f} ({mode:.1%})")

# 95% credible interval
lower = stats.beta.ppf(0.025, alpha, beta)
upper = stats.beta.ppf(0.975, alpha, beta)

print(f"\n95% credible interval: [{lower:.3f}, {upper:.3f}]")
print(f"95% credible interval: [{lower:.1%}, {upper:.1%}]")

In [ ]:
# Plot Beta PDFs for different shapes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

x = np.linspace(0, 1, 1000)

# Different parameter combinations
param_sets = [
    [(1, 1), (2, 2), (5, 5)],  # Symmetric
    [(0.5, 0.5), (2, 5), (5, 2)],  # Various shapes
    [(2, 5), (10, 25), (20, 50)],  # Same mean, different concentrations
    [(1, 3), (2, 6), (10, 30)]  # Same mean, increasing samples
]

titles = [
    'Symmetric Beta Distributions',
    'Various Beta Shapes',
    'Same Mean (0.286), Different Concentrations',
    'Same Mean (0.25), Different Sample Sizes'
]

for ax, params, title in zip(axes.flat, param_sets, titles):
    for alpha, beta in params:
        pdf = stats.beta.pdf(x, alpha, beta)
        mean = alpha / (alpha + beta)
        ax.plot(x, pdf, linewidth=2, 
                label=f'α={alpha}, β={beta} (mean={mean:.2f})')
    
    ax.set_xlabel('Probability (p)')
    ax.set_ylabel('Probability Density')
    ax.set_title(title)
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insights:")
print("  - α = β gives symmetric distribution")
print("  - Larger α shifts distribution right")
print("  - Larger α + β gives tighter (more concentrated) distribution")
print("  - Beta(1,1) is uniform distribution")

In [ ]:
# Simulation: Beta samples
print("Simulation: Beta Distribution Samples")
print("=" * 40)

alpha = 8
beta = 4
n_samples = 10000

# Generate samples
samples = np.random.beta(alpha, beta, n_samples)

print(f"Generating {n_samples} samples from Beta({alpha}, {beta})")
print(f"\nSimulated mean: {np.mean(samples):.4f}")
print(f"Theoretical mean: {alpha/(alpha+beta):.4f}")
print(f"\nSimulated std: {np.std(samples, ddof=1):.4f}")
theoretical_std = np.sqrt((alpha*beta)/((alpha+beta)**2*(alpha+beta+1)))
print(f"Theoretical std: {theoretical_std:.4f}")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(samples, bins=50, density=True, alpha=0.7, 
        color='orange', edgecolor='black', label='Simulated data')

x_range = np.linspace(0, 1, 1000)
pdf_theoretical = stats.beta.pdf(x_range, alpha, beta)
ax.plot(x_range, pdf_theoretical, 'r-', linewidth=2, 
        label='Theoretical PDF')

ax.axvline(alpha/(alpha+beta), color='green', linestyle='--', 
           linewidth=2, label=f'Mean = {alpha/(alpha+beta):.2f}')

ax.set_xlabel('Probability (p)')
ax.set_ylabel('Probability Density')
ax.set_title(f'Beta Distribution: Simulated vs Theoretical (α={alpha}, β={beta})')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Exercise: Shooting Percentage with Bayesian Updates

We'll model a player's shooting percentage using Beta distribution.

**Bayesian update rule:**
- Prior: Beta(α, β)
- Observe: m makes, n misses
- Posterior: Beta(α + m, β + n)

In [ ]:
# Exercise: Bayesian shooting percentage
print("Exercise: Bayesian Shooting Percentage")
print("=" * 40)

# Start with weakly informative prior
# Beta(2,2) centered at 0.5 with some uncertainty
alpha_prior = 2
beta_prior = 2

print("Prior: Beta(2, 2)")
print(f"Prior mean: {alpha_prior/(alpha_prior+beta_prior):.3f}")

# Player shoots 10 times: 7 makes, 3 misses
makes = 7
misses = 3

print(f"\nObserved data: {makes} makes, {misses} misses")
print(f"Sample shooting %: {makes/(makes+misses):.3f}")

# Update to posterior
alpha_post = alpha_prior + makes
beta_post = beta_prior + misses

print(f"\nPosterior: Beta({alpha_post}, {beta_post})")
print(f"Posterior mean: {alpha_post/(alpha_post+beta_post):.3f}")

# Additional shots: 15 more makes, 5 more misses
makes2 = 15
misses2 = 5

print(f"\nMore data: {makes2} makes, {misses2} misses")

# Update again
alpha_post2 = alpha_post + makes2
beta_post2 = beta_post + misses2

print(f"\nUpdated posterior: Beta({alpha_post2}, {beta_post2})")
print(f"Updated posterior mean: {alpha_post2/(alpha_post2+beta_post2):.3f}")

# Plot evolution of beliefs
fig, ax = plt.subplots(figsize=(10, 6))

x = np.linspace(0, 1, 1000)

# Prior
pdf_prior = stats.beta.pdf(x, alpha_prior, beta_prior)
ax.plot(x, pdf_prior, linewidth=2, label='Prior', alpha=0.7)

# First posterior
pdf_post1 = stats.beta.pdf(x, alpha_post, beta_post)
ax.plot(x, pdf_post1, linewidth=2, label=f'After {makes+misses} shots', alpha=0.7)

# Second posterior
pdf_post2 = stats.beta.pdf(x, alpha_post2, beta_post2)
ax.plot(x, pdf_post2, linewidth=2, label=f'After {makes+misses+makes2+misses2} shots', alpha=0.7)

# Add vertical lines for means
ax.axvline(alpha_prior/(alpha_prior+beta_prior), color='C0', 
           linestyle='--', alpha=0.5)
ax.axvline(alpha_post/(alpha_post+beta_post), color='C1', 
           linestyle='--', alpha=0.5)
ax.axvline(alpha_post2/(alpha_post2+beta_post2), color='C2', 
           linestyle='--', alpha=0.5)

ax.set_xlabel('Shooting Percentage')
ax.set_ylabel('Probability Density')
ax.set_title('Bayesian Update of Shooting Percentage')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: As we observe more data, the distribution")
print("becomes more concentrated around the true shooting percentage.")

---

## 4. Central Limit Theorem

### What is the Central Limit Theorem?

The Central Limit Theorem (CLT) is one of the most important results in statistics.

**Statement:** When you take the mean of many independent random samples, the distribution of those means approaches a normal distribution, regardless of the original distribution.

**Key points:**
- Works for ANY original distribution
- Sample size matters (larger is better)
- Usually n ≥ 30 is sufficient
- The mean of sample means equals the population mean
- The standard deviation of sample means = σ/√n

### Simple Example

**Problem:** Roll a fair die many times. Each roll gives 1, 2, 3, 4, 5, or 6 (uniform, not normal). Take the average of 10 rolls. Repeat this 1000 times. What does the distribution of these averages look like?

**Answer:** Even though individual rolls are uniform, the distribution of averages will be approximately normal!

In [ ]:
# Simple CLT example with dice
print("Simple CLT Example: Dice Rolls")
print("=" * 40)

# Single die: uniform distribution
die_values = [1, 2, 3, 4, 5, 6]
pop_mean = np.mean(die_values)
pop_std = np.std(die_values, ddof=0)

print(f"Population (single die):")
print(f"  Mean: {pop_mean:.2f}")
print(f"  Std: {pop_std:.2f}")

# Sample size
n = 10
n_samples = 1000

# Generate sample means
sample_means = []
for _ in range(n_samples):
    # Roll die n times and take average
    rolls = np.random.choice(die_values, size=n)
    sample_means.append(np.mean(rolls))

sample_means = np.array(sample_means)

print(f"\nSample means (average of {n} rolls):")
print(f"  Mean: {np.mean(sample_means):.2f}")
print(f"  Std: {np.std(sample_means, ddof=1):.2f}")
print(f"  Expected std (σ/√n): {pop_std/np.sqrt(n):.2f}")

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Original distribution (uniform)
ax1.hist(die_values, bins=6, density=True, alpha=0.7, 
         color='skyblue', edgecolor='black', rwidth=0.8)
ax1.set_xlabel('Die Value')
ax1.set_ylabel('Probability')
ax1.set_title('Original Distribution (Uniform)')
ax1.set_xticks(die_values)
ax1.grid(alpha=0.3)

# Distribution of sample means (approximately normal)
ax2.hist(sample_means, bins=30, density=True, alpha=0.7, 
         color='coral', edgecolor='black')

# Overlay theoretical normal
x_range = np.linspace(np.min(sample_means), np.max(sample_means), 1000)
pdf_normal = stats.norm.pdf(x_range, pop_mean, pop_std/np.sqrt(n))
ax2.plot(x_range, pdf_normal, 'r-', linewidth=2, 
         label='Theoretical Normal')

ax2.set_xlabel('Sample Mean')
ax2.set_ylabel('Probability Density')
ax2.set_title(f'Distribution of Sample Means (n={n})')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nCLT in action: Uniform → Normal through averaging!")

### Demonstration with Non-Normal Distribution

Let's use an exponential distribution (highly skewed) and show how sample means become normal.

In [ ]:
# CLT with exponential distribution
print("CLT Demonstration: Exponential Distribution")
print("=" * 40)

# Population: Exponential(λ=1)
lambda_param = 1
pop_mean = 1 / lambda_param
pop_std = 1 / lambda_param

print(f"Population (Exponential):")
print(f"  Mean: {pop_mean:.2f}")
print(f"  Std: {pop_std:.2f}")

# Different sample sizes
sample_sizes = [2, 5, 10, 30]
n_samples = 5000

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, n in zip(axes.flat, sample_sizes):
    # Generate sample means
    sample_means = []
    for _ in range(n_samples):
        sample = np.random.exponential(1/lambda_param, n)
        sample_means.append(np.mean(sample))
    
    sample_means = np.array(sample_means)
    
    # Plot histogram
    ax.hist(sample_means, bins=40, density=True, alpha=0.7, 
            color='lightgreen', edgecolor='black')
    
    # Overlay theoretical normal
    x_range = np.linspace(np.min(sample_means), np.max(sample_means), 1000)
    pdf_normal = stats.norm.pdf(x_range, pop_mean, pop_std/np.sqrt(n))
    ax.plot(x_range, pdf_normal, 'r-', linewidth=2, 
            label='Theoretical Normal')
    
    ax.set_xlabel('Sample Mean')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Sample Size n = {n}')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Print statistics
    print(f"\nn = {n}:")
    print(f"  Empirical mean: {np.mean(sample_means):.3f}")
    print(f"  Empirical std: {np.std(sample_means, ddof=1):.3f}")
    print(f"  Expected std: {pop_std/np.sqrt(n):.3f}")

plt.suptitle('Central Limit Theorem: Sample Means from Exponential Distribution', 
             fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

print("\nKey insight: Larger n → closer to normal distribution!")

### Exercise: Basketball Game Averages

A player's points per game follow some distribution (not normal).

We'll simulate averaging over 10 games and show convergence to normality.

In [ ]:
# Exercise: Basketball scoring averages
print("Exercise: Basketball Game Averages")
print("=" * 40)

# Simulate points per game with a skewed distribution
# Use Gamma(k=4, θ=5) for points
k = 4
theta = 5
pop_mean = k * theta
pop_std = np.sqrt(k) * theta

print(f"True distribution: Gamma({k}, {theta})")
print(f"Population mean: {pop_mean} points")
print(f"Population std: {pop_std:.2f} points")

# Sample individual games
individual_games = np.random.gamma(k, theta, 10000)

# Different numbers of games to average
games_to_average = [1, 5, 10, 20]
n_players = 2000  # Number of "players" to simulate

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, n_games in zip(axes.flat, games_to_average):
    # For each player, average n_games
    averages = []
    for _ in range(n_players):
        games = np.random.gamma(k, theta, n_games)
        averages.append(np.mean(games))
    
    averages = np.array(averages)
    
    # Plot histogram
    ax.hist(averages, bins=40, density=True, alpha=0.7, 
            color='purple', edgecolor='black')
    
    # Overlay theoretical normal (by CLT)
    x_range = np.linspace(np.min(averages), np.max(averages), 1000)
    pdf_normal = stats.norm.pdf(x_range, pop_mean, pop_std/np.sqrt(n_games))
    ax.plot(x_range, pdf_normal, 'r-', linewidth=2, 
            label='Normal approximation')
    
    # If n_games=1, also show original Gamma
    if n_games == 1:
        pdf_gamma = stats.gamma.pdf(x_range, k, scale=theta)
        ax.plot(x_range, pdf_gamma, 'g--', linewidth=2, 
                label='Original Gamma')
    
    ax.set_xlabel('Average Points')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Average over {n_games} game(s)')
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle('CLT: Averaging Basketball Scoring', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

print("\nObservation: As we average more games, the distribution")
print("becomes more normal and less variable.")

---

## 5. Maximum Likelihood Estimation (MLE)

### What is MLE?

Maximum Likelihood Estimation is a method to estimate parameters of a distribution.

**Core idea:** Choose the parameter value that makes the observed data most likely.

**Process:**
1. Write down the likelihood function (probability of data given parameter)
2. Find the parameter that maximizes this likelihood
3. Usually maximize log-likelihood (easier math)

**Why it works:** The best parameter is the one that best explains what we actually observed.

### MLE for Exponential Distribution

**Setup:** We observe waiting times x₁, x₂, ..., xₙ from Exponential(λ).

**Goal:** Estimate λ

**Likelihood function:**
$$L(\lambda) = \prod_{i=1}^{n} \lambda e^{-\lambda x_i} = \lambda^n e^{-\lambda \sum x_i}$$

**Log-likelihood:**
$$\log L(\lambda) = n \log(\lambda) - \lambda \sum_{i=1}^{n} x_i$$

**Derivative:**
$$\frac{d}{d\lambda} \log L = \frac{n}{\lambda} - \sum_{i=1}^{n} x_i$$

**Set to zero and solve:**
$$\frac{n}{\lambda} = \sum_{i=1}^{n} x_i$$

**MLE estimate:**
$$\hat{\lambda} = \frac{n}{\sum_{i=1}^{n} x_i} = \frac{1}{\bar{x}}$$

The MLE for λ is the reciprocal of the sample mean!

In [ ]:
# MLE for Exponential: Example
print("MLE for Exponential Distribution")
print("=" * 40)

# True parameter
true_lambda = 3

# Generate data
np.random.seed(42)
n = 100
data = np.random.exponential(1/true_lambda, n)

print(f"True λ: {true_lambda}")
print(f"Sample size: {n}")
print(f"\nData summary:")
print(f"  Sample mean: {np.mean(data):.4f}")
print(f"  Expected mean (1/λ): {1/true_lambda:.4f}")

# MLE estimate
lambda_mle = 1 / np.mean(data)

print(f"\nMLE estimate: λ̂ = {lambda_mle:.4f}")
print(f"True value: λ = {true_lambda}")
print(f"Error: {abs(lambda_mle - true_lambda):.4f}")

# Plot likelihood function
lambda_values = np.linspace(1, 5, 1000)
log_likelihoods = []

for lam in lambda_values:
    # Log-likelihood: n*log(λ) - λ*sum(x_i)
    log_lik = n * np.log(lam) - lam * np.sum(data)
    log_likelihoods.append(log_lik)

log_likelihoods = np.array(log_likelihoods)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(lambda_values, log_likelihoods, 'b-', linewidth=2)
ax.axvline(lambda_mle, color='red', linestyle='--', linewidth=2, 
           label=f'MLE: λ̂ = {lambda_mle:.2f}')
ax.axvline(true_lambda, color='green', linestyle='--', linewidth=2, 
           label=f'True: λ = {true_lambda}')

ax.set_xlabel('λ')
ax.set_ylabel('Log-Likelihood')
ax.set_title('Log-Likelihood Function for Exponential Distribution')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### MLE for Poisson Distribution

**Setup:** We observe counts x₁, x₂, ..., xₙ from Poisson(λ).

**Goal:** Estimate λ

**Likelihood function:**
$$L(\lambda) = \prod_{i=1}^{n} \frac{\lambda^{x_i} e^{-\lambda}}{x_i!}$$

**Log-likelihood:**
$$\log L(\lambda) = \sum_{i=1}^{n} x_i \log(\lambda) - n\lambda - \sum_{i=1}^{n} \log(x_i!)$$

**Derivative:**
$$\frac{d}{d\lambda} \log L = \frac{\sum_{i=1}^{n} x_i}{\lambda} - n$$

**Set to zero and solve:**
$$\frac{\sum_{i=1}^{n} x_i}{\lambda} = n$$

**MLE estimate:**
$$\hat{\lambda} = \frac{\sum_{i=1}^{n} x_i}{n} = \bar{x}$$

The MLE for λ is simply the sample mean!

In [ ]:
# MLE for Poisson: Example
print("MLE for Poisson Distribution")
print("=" * 40)

# True parameter
true_lambda = 5

# Generate data
np.random.seed(42)
n = 100
data = np.random.poisson(true_lambda, n)

print(f"True λ: {true_lambda}")
print(f"Sample size: {n}")
print(f"\nData summary:")
print(f"  Sample mean: {np.mean(data):.4f}")
print(f"  Sample variance: {np.var(data, ddof=1):.4f}")
print(f"  (For Poisson, mean = variance = λ)")

# MLE estimate
lambda_mle = np.mean(data)

print(f"\nMLE estimate: λ̂ = {lambda_mle:.4f}")
print(f"True value: λ = {true_lambda}")
print(f"Error: {abs(lambda_mle - true_lambda):.4f}")

# Plot likelihood function
lambda_values = np.linspace(3, 7, 1000)
log_likelihoods = []

for lam in lambda_values:
    # Log-likelihood: sum(x_i)*log(λ) - n*λ - sum(log(x_i!))
    log_lik = np.sum(data) * np.log(lam) - n * lam - np.sum(np.log(np.array([np.math.factorial(x) for x in data])))
    log_likelihoods.append(log_lik)

log_likelihoods = np.array(log_likelihoods)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(lambda_values, log_likelihoods, 'b-', linewidth=2)
ax.axvline(lambda_mle, color='red', linestyle='--', linewidth=2, 
           label=f'MLE: λ̂ = {lambda_mle:.2f}')
ax.axvline(true_lambda, color='green', linestyle='--', linewidth=2, 
           label=f'True: λ = {true_lambda}')

ax.set_xlabel('λ')
ax.set_ylabel('Log-Likelihood')
ax.set_title('Log-Likelihood Function for Poisson Distribution')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Exercise: MLE for Basketball Scoring

We'll use real-world-like basketball scoring data.

Points scored per quarter can be modeled as Poisson.

We'll estimate λ using MLE and compare with the true value.

In [ ]:
# Exercise: Basketball scoring MLE
print("Exercise: MLE for Basketball Scoring")
print("=" * 40)

# Simulate points per quarter
# Typical team scores about 25-30 points per quarter
true_lambda = 27  # Average points per quarter

# Simulate a season (82 games, 4 quarters each)
np.random.seed(123)
n_games = 82
quarters_per_game = 4
n_quarters = n_games * quarters_per_game

points_per_quarter = np.random.poisson(true_lambda, n_quarters)

print(f"True λ (average points per quarter): {true_lambda}")
print(f"Number of quarters: {n_quarters}")

# MLE estimate
lambda_mle = np.mean(points_per_quarter)

print(f"\nMLE estimate: λ̂ = {lambda_mle:.2f} points per quarter")
print(f"True value: λ = {true_lambda} points per quarter")
print(f"Error: {abs(lambda_mle - true_lambda):.2f} points")
print(f"Relative error: {abs(lambda_mle - true_lambda)/true_lambda * 100:.2f}%")

# Visualize the data vs. model
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of observed data
counts, bins, _ = ax1.hist(points_per_quarter, bins=20, density=True, 
                           alpha=0.7, color='skyblue', edgecolor='black',
                           label='Observed data')

# Overlay fitted Poisson PMF
x_vals = np.arange(10, 45)
pmf_fitted = stats.poisson.pmf(x_vals, lambda_mle)
ax1.plot(x_vals, pmf_fitted, 'ro-', linewidth=2, markersize=4,
         label=f'Fitted Poisson(λ̂={lambda_mle:.1f})')

# True Poisson PMF
pmf_true = stats.poisson.pmf(x_vals, true_lambda)
ax1.plot(x_vals, pmf_true, 'g^--', linewidth=2, markersize=4,
         label=f'True Poisson(λ={true_lambda})')

ax1.set_xlabel('Points per Quarter')
ax1.set_ylabel('Probability')
ax1.set_title('Points Distribution: Data vs Model')
ax1.legend()
ax1.grid(alpha=0.3)

# Log-likelihood surface
lambda_range = np.linspace(24, 30, 1000)
log_liks = []

for lam in lambda_range:
    log_lik = np.sum(stats.poisson.logpmf(points_per_quarter, lam))
    log_liks.append(log_lik)

ax2.plot(lambda_range, log_liks, 'b-', linewidth=2)
ax2.axvline(lambda_mle, color='red', linestyle='--', linewidth=2,
           label=f'MLE: λ̂={lambda_mle:.2f}')
ax2.axvline(true_lambda, color='green', linestyle='--', linewidth=2,
           label=f'True: λ={true_lambda}')

ax2.set_xlabel('λ (points per quarter)')
ax2.set_ylabel('Log-Likelihood')
ax2.set_title('Log-Likelihood Function')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nConclusion: MLE provides an excellent estimate of the")
print("true scoring rate from observed data!")

---

## Summary

### When to Use Each Distribution

**Exponential Distribution:**
- Time between independent events
- Waiting times with constant rate
- Memoryless process
- Examples: time between arrivals, time to failure

**Gamma Distribution:**
- Sum of multiple exponential waiting times
- Time until k events occur
- Positive continuous data with some skewness
- Examples: time until multiple goals, sum of waiting times

**Beta Distribution:**
- Probabilities or proportions (values between 0 and 1)
- Prior/posterior distributions for rates
- Modeling uncertainty about probabilities
- Examples: shooting percentages, conversion rates, win probabilities

**Central Limit Theorem:**
- Not a distribution, but a fundamental theorem
- Justifies using normal approximations for sample means
- Critical for hypothesis testing and confidence intervals
- Works regardless of original distribution (if n is large enough)

**Maximum Likelihood Estimation:**
- General method for parameter estimation
- Finds parameter values that best explain observed data
- Works for any parametric distribution
- Provides optimal estimates under mild conditions

### Quick Reference Table

| Distribution | Type | Parameters | Mean | Variance | Support |
|--------------|------|------------|------|----------|----------|
| Exponential | Continuous | λ (rate) | 1/λ | 1/λ² | [0, ∞) |
| Gamma | Continuous | k (shape), θ (scale) | kθ | kθ² | (0, ∞) |
| Beta | Continuous | α, β (shapes) | α/(α+β) | αβ/[(α+β)²(α+β+1)] | [0, 1] |

---

## Practice Problems

### Problem 1: Exponential
Goals in a soccer game occur at a rate of 0.5 per hour (λ = 0.5). What is the probability that the first goal occurs within 30 minutes (0.5 hours)?

**Answer:** 0.221 (or 22.1%)

### Problem 2: Gamma
If goals occur at rate λ = 0.5 per hour, what is the expected time until 3 goals are scored?

**Answer:** 6 hours (mean = k/λ = 3/0.5)

### Problem 3: Beta
A player has Beta(20, 10) distribution for their shooting percentage. What is their expected shooting percentage?

**Answer:** 0.667 (or 66.7%)

### Problem 4: Central Limit Theorem
A die has mean 3.5 and standard deviation 1.71. If you roll it 36 times and take the average, what is the standard deviation of the sample mean?

**Answer:** 0.285 (σ/√n = 1.71/6)

### Problem 5: MLE
You observe waiting times: [2.1, 0.8, 1.5, 3.2, 0.9] from an exponential distribution. What is the MLE estimate of λ?

**Answer:** λ̂ = 1/mean = 1/1.7 ≈ 0.588

In [ ]:
# Solutions to practice problems
print("PRACTICE PROBLEM SOLUTIONS")
print("=" * 50)

# Problem 1
print("\nProblem 1: Exponential")
lambda_rate = 0.5
time = 0.5
prob = stats.expon.cdf(time, scale=1/lambda_rate)
print(f"Answer: {prob:.3f} or {prob:.1%}")

# Problem 2
print("\nProblem 2: Gamma")
k = 3
lambda_rate = 0.5
mean = k / lambda_rate
print(f"Answer: {mean} hours")

# Problem 3
print("\nProblem 3: Beta")
alpha = 20
beta = 10
mean = alpha / (alpha + beta)
print(f"Answer: {mean:.3f} or {mean:.1%}")

# Problem 4
print("\nProblem 4: Central Limit Theorem")
sigma = 1.71
n = 36
std_sample_mean = sigma / np.sqrt(n)
print(f"Answer: {std_sample_mean:.3f}")

# Problem 5
print("\nProblem 5: MLE")
waiting_times = np.array([2.1, 0.8, 1.5, 3.2, 0.9])
lambda_mle = 1 / np.mean(waiting_times)
print(f"Answer: λ̂ = {lambda_mle:.3f}")

---

## Conclusion

Congratulations! You have learned about advanced probability concepts:

1. **Exponential Distribution** - modeling waiting times
2. **Gamma Distribution** - sum of exponential waiting times
3. **Beta Distribution** - modeling probabilities and proportions
4. **Central Limit Theorem** - why averages are approximately normal
5. **Maximum Likelihood Estimation** - estimating parameters from data

These tools are essential for statistical inference, machine learning, and data science. Practice applying them to real-world problems to build deeper intuition!